In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as mat
import datetime as dt

In [2]:
# Read the datafiles
data_path = '../data/'
data_file_list = ['turnstile_191102.txt',
                 'turnstile_191109.txt',
                 'turnstile_191116.txt',
                 'turnstile_191130.txt',
                 'turnstile_191207.txt',
                 'turnstile_191214.txt',
                 'turnstile_191221.txt',
                 'turnstile_191228.txt'
                 'turnstile_200104.txt']

booth_key_filename = 'Remote-Booth-Station.csv'

In [4]:
turnstiles = pd.read_csv(data_path + data_file_list[0])
booth_key = pd.read_csv(data_path + booth_key_filename)
turnstiles, booth_key

FileNotFoundError: [Errno 2] File b'../data/turnstile_191102.txt' does not exist: b'../data/turnstile_191102.txt'

In [4]:
turnstiles.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679


In [5]:
turnstiles.columns = [x.strip() for x in turnstiles.columns]
turnstiles.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [6]:
turnstiles.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

In [7]:
delta_entry = [0]
delta_exit = [0]
date_list = [dt.datetime.strptime(turnstiles['date'][0], '%m/%d/%Y')]
day_list = [date_list[0].weekday()]
for row in range(1, len(turnstiles)) :
    if((turnstiles["station"][row] == turnstiles['station'][row - 1]) &
        (turnstiles['entries'][row] > turnstiles['entries'][row - 1])):
        delta_entry.append(turnstiles['entries'][row] - turnstiles['entries'][row - 1])
        delta_exit.append(turnstiles['exits'][row] - turnstiles['exits'][row - 1])
    else :
        delta_entry.append(0)
        delta_exit.append(0)
                           
    date_list.append(dt.datetime.strptime(turnstiles['date'][row], '%m/%d/%Y'))
    day_list.append(date_list[row].weekday())
                 

turnstiles['deltaEntry'] = delta_entry
turnstiles['deltaExit'] = delta_exit
turnstiles['datetime'] = date_list
turnstiles['weekday'] = day_list;

In [8]:
turnstiles.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,deltaEntry,deltaExit,datetime,weekday
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491,0,0,2019-10-26,5
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499,14,8,2019-10-26,5
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532,15,33,2019-10-26,5
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623,112,91,2019-10-26,5
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679,292,56,2019-10-26,5


In [9]:
turnstiles.groupby(['station', 'time'])[['deltaEntry', 'deltaExit']].sum()

deltaEntry   deltaExit
station   time                            
1 AV      00:00:00   359596754   370469565
          04:00:00   957759927  1013566881
          08:00:00        7455       15566
          12:00:00       25122       28529
          16:00:00       25183       21914
...                        ...         ...
ZEREGA AV 05:00:00         426         557
          09:00:00        6879        2805
          13:00:00        3206        2116
          17:00:00        3793        3802
          21:00:00        2105        4732

[16020 rows x 2 columns]

In [10]:
import folium

In [11]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [12]:
basemap = generateBaseMap()
basemap

In [13]:
station_data = 'NYC_Transit_Subway_Entrance_And_Exit_Data.csv'
stations = pd.read_csv(data_path+station_data)

In [14]:
BBox = (stations['Entrance Longitude'].min(),   stations['Entrance Longitude'].max(),      
         stations['Entrance Latitude'].min(), stations['Entrance Latitude'].max())
BBox

(-74.03138299999999, 73.99332700000001, 40.575499, 40.903597)

In [15]:
stations.describe()

,Station Latitude,Station Longitude,Route8,Route9,Route10,Route11,Entrance Latitude,Entrance Longitude
count,1868.000000,1868.000000,48.000000,28.000000,23.0,23.0,1868.000000,1868.000000
mean,40.734065,-73.943609,2.979167,2.535714,3.0,7.0,40.734063,-73.864370
std,0.070376,0.057230,1.940493,1.170063,0.0,0.0,0.070382,3.423331
min,40.576127,-74.030876,1.000000,2.000000,3.0,7.0,40.575499,-74.031383
25%,40.689941,-73.987495,1.000000,2.000000,3.0,7.0,40.689808,-73.987482
50%,40.733422,-73.958145,4.000000,2.000000,3.0,7.0,40.733535,-73.958385
75%,40.768247,-73.911794,5.000000,2.000000,3.0,7.0,40.767655,-73.911159
max,40.903125,-73.755405,5.000000,5.000000,3.0,7.0,40.903597,73.993327


In [16]:
turnstiles.describe()

,entries,exits,deltaEntry,deltaExit,weekday
count,2.060460e+05,2.060460e+05,2.060460e+05,2.060460e+05,206046.000000
mean,4.101341e+07,3.395214e+07,7.233035e+05,5.445642e+05,2.993186
std,2.113375e+08,1.958360e+08,2.871428e+07,2.615529e+07,1.999556
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.960513e+09,0.000000
25%,2.987312e+05,1.387082e+05,8.000000e+00,6.000000e+00,1.000000
50%,1.969911e+06,1.157290e+06,7.300000e+01,5.000000e+01,3.000000
75%,6.621930e+06,4.505968e+06,2.490000e+02,1.710000e+02,5.000000
max,2.129010e+09,2.123659e+09,2.120528e+09,2.117985e+09,6.000000


In [17]:
stations.head()

,Division,Line,Station Name,Station Latitude,Station Longitude,Route1,Route2,Route3,Route4,Route5,...,ADA,ADA Notes,Free Crossover,North South Street,East West Street,Corner,Entrance Latitude,Entrance Longitude,Station Location,Entrance Location
0,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SW,40.660489,-73.998220,"(40.660397, -73.998091)","(40.660489, -73.99822)"
1,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SE,40.660323,-73.997952,"(40.660397, -73.998091)","(40.660323, -73.997952)"
2,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654676,-74.004306,"(40.655144, -74.003549)","(40.654676, -74.004306)"
3,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NE,40.654365,-74.004113,"(40.655144, -74.003549)","(40.654365, -74.004113)"
4,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654490,-74.004499,"(40.655144, -74.003549)","(40.65449, -74.004499)"


In [18]:
stations.rename(columns=lambda x: x.strip().lower().replace(' ', '_'), inplace=True)

In [19]:
stations.station_name.unique()

array(['25th St', '36th St', '45th St', '53rd St', '59th St', '77th St',
       '86th St', '95th St', '9th St', 'Atlantic Av-Barclays Ctr',
       'Bay Ridge Av', 'DeKalb Av', 'Pacific St', 'Prospect Av',
       'Union St', '30 Av-Grand Av', '36 Av-Washington Av',
       '39 Av-Beebe Av', 'Astoria Blvd-Hoyt Av', 'Broadway',
       'Ditmars Blvd', '7th Av', 'Atlantic Av', 'Av H', 'Av J', 'Av M',
       'Av U', 'Beverly Rd', 'Brighton Beach', 'Church Av',
       'Cortelyou Rd', 'Kings Highway', 'Neck Rd', 'Newkirk Av',
       'Ocean Parkway', 'Parkside Av', 'Prospect Park', 'Sheepshead Bay',
       'Stillwell Av', 'West 8th St', '23rd St', '28th St', '34th St',
       '49th St', '57th St', '5th Av', '8th St', 'Canal St (UL)',
       'City Hall', 'Cortlandt St', 'Court St', 'Lawrence St',
       'Lexington Av', 'Prince St', 'Rector St', 'Times Square-42nd St',
       'Union Square', 'Whitehall St', '104th St-102nd St', '111th St',
       '121st St', 'Alabama Av', 'Chauncey St', 'Cleveland

In [20]:
len(stations.station_name)

1868

In [21]:
stations.head()

,division,line,station_name,station_latitude,station_longitude,route1,route2,route3,route4,route5,...,ada,ada_notes,free_crossover,north_south_street,east_west_street,corner,entrance_latitude,entrance_longitude,station_location,entrance_location
0,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SW,40.660489,-73.998220,"(40.660397, -73.998091)","(40.660489, -73.99822)"
1,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SE,40.660323,-73.997952,"(40.660397, -73.998091)","(40.660323, -73.997952)"
2,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654676,-74.004306,"(40.655144, -74.003549)","(40.654676, -74.004306)"
3,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NE,40.654365,-74.004113,"(40.655144, -74.003549)","(40.654365, -74.004113)"
4,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654490,-74.004499,"(40.655144, -74.003549)","(40.65449, -74.004499)"


In [22]:
stations.division.value_counts()

IND    726
IRT    700
BMT    442
Name: division, dtype: int64

In [56]:
from folium.plugins import HeatMap
stations_copy = stations.copy()
stations_copy['count'] = 1
basemap = generateBaseMap()
HeatMap(data=stations_copy[['station_latitude', 'station_longitude', 'count']].groupby(['station_latitude', 'station_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(basemap)

In [57]:
basemap

In [25]:
less_stations = stations_copy[stations_copy['division'] == 'BMT']

In [58]:
for lat,lon,station in zip(less_stations['station_latitude'],less_stations['station_longitude'],less_stations['station_name']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='b',
                            fill=True,
                            fill_opacity=0.7,
                           ).add_to(basemap)

In [59]:
basemap

## Merge turnstiles and station data

In [32]:
turnstiles_copy = turnstiles.copy()

In [33]:
turnstiles_copy.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,deltaEntry,deltaExit,datetime,weekday
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491,0,0,2019-10-26,5
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499,14,8,2019-10-26,5
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532,15,33,2019-10-26,5
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623,112,91,2019-10-26,5
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679,292,56,2019-10-26,5


Clean station names to match `stations` DataFrame

In [42]:
turnstiles_copy['station'] = turnstiles_copy['station'].str.title()

Filter down the DataFrames to only include relevant data

In [61]:
turnstiles_copy = turnstiles_copy.loc[:, ['station', 'division', 'date', 'time', 'entries', 'exits', 'deltaEntry', 'deltaExit', 'datetime', 'weekday']]

In [62]:
turnstiles_copy.head()

,station,division,date,time,entries,exits,deltaEntry,deltaExit,datetime,weekday
0,59 St,BMT,10/26/2019,00:00:00,7247322,2455491,0,0,2019-10-26,5
1,59 St,BMT,10/26/2019,04:00:00,7247336,2455499,14,8,2019-10-26,5
2,59 St,BMT,10/26/2019,08:00:00,7247351,2455532,15,33,2019-10-26,5
3,59 St,BMT,10/26/2019,12:00:00,7247463,2455623,112,91,2019-10-26,5
4,59 St,BMT,10/26/2019,16:00:00,7247755,2455679,292,56,2019-10-26,5


Convert stations with numerics to ordinal numbers

In [87]:
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])
print([ordinal(n) for n in range(1, 32)])

['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11st', '12nd', '13rd', '14th', '15th', '16th', '17th', '18th', '19th', '20th', '21st', '22nd', '23rd', '24th', '25th', '26th', '27th', '28th', '29th', '30th', '31st']


In [91]:
for x in turnstiles_copy.station:
    if any(char.isdigit() for char in x):
        ordinal(x)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

# COME BACK TO ABOVE TOMORROW AM TO GET MERGE RIGHT

In [63]:
stations_copy = stations_copy.loc[:, ['division', 'station_name', 'station_latitude', 'station_longitude']]

In [64]:
stations_copy.head()

,division,station_name,station_latitude,station_longitude
0,BMT,25th St,40.660397,-73.998091
1,BMT,25th St,40.660397,-73.998091
2,BMT,36th St,40.655144,-74.003549
3,BMT,36th St,40.655144,-74.003549
4,BMT,36th St,40.655144,-74.003549


In [70]:
stations_grouped = stations_copy.groupby(['station_name', 'division'], as_index=False)['station_latitude', 'station_longitude'].mean()

Merge DataFrames on station

In [82]:
merged_data = pd.merge(turnstiles_copy, stations_grouped, left_on='station', right_on='station_name', how='inner')

In [83]:
merged_data.columns

Index(['station', 'division_x', 'date', 'time', 'entries', 'exits',
       'deltaEntry', 'deltaExit', 'datetime', 'weekday', 'station_name',
       'division_y', 'station_latitude', 'station_longitude'],
      dtype='object')

In [84]:
merged_data.head()

,station,division_x,date,time,entries,exits,deltaEntry,deltaExit,datetime,weekday,station_name,division_y,station_latitude,station_longitude
0,Prince St,BMT,10/26/2019,00:00:00,12310132,4648104,0,0,2019-10-26,5,Prince St,BMT,40.724329,-73.997702
1,Prince St,BMT,10/26/2019,04:00:00,12310171,4648108,39,4,2019-10-26,5,Prince St,BMT,40.724329,-73.997702
2,Prince St,BMT,10/26/2019,08:00:00,12310177,4648132,6,24,2019-10-26,5,Prince St,BMT,40.724329,-73.997702
3,Prince St,BMT,10/26/2019,12:00:00,12310292,4648263,115,131,2019-10-26,5,Prince St,BMT,40.724329,-73.997702
4,Prince St,BMT,10/26/2019,16:00:00,12310875,4648594,583,331,2019-10-26,5,Prince St,BMT,40.724329,-73.997702


In [85]:
merged_data.shape

(90469, 14)